This notebook will display the code used to develop models that recognize user intent in Spanish and English. 

# Libraries

Here are the required libraries for the model. The details for installing them are in the read.me and the text_requirements.txt files.

In [2]:
import json
import nltk
import spacy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from textblob import TextBlob
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from textblob import TextBlob
import os
from autocorrect import Speller
from spellchecker import SpellChecker
import dateparser

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


C:\Users\aashi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\aashi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\aashi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\aashi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

# Load JSON Files

In [3]:
# English Data
data_en = json.loads(open('data_en.json').read())

# Spanish Data
data_es = json.loads(open('data_es.json').read())

## Natural Language Processing

In this section, we input all of the data that was loaded and derive lists of intents and their respective speech patterns with the NLTK library. 

In [4]:
intents_en = data_en['interactionModel']['languageModel']

intents_es = data_es['interactionModel']['languageModel']


In [5]:
# Create a list of tuples that assigns a list of words to an intent in English
words_en = []
labels_en = []
documents_en = []
punc_en = ['?', '!', ',', '.']

for intent in intents_en['intent']: 
    for sample in intent['samples']:
        word_list_en = nltk.word_tokenize(sample)
        words_en.extend(word_list_en)
        documents_en.append((word_list_en, intent['name']))
        if intent['name'] not in labels_en:
            labels_en.append(intent['name'])


In [6]:
# Create a list of tuples that assigns a list of words to an intent in Spanish
words_es = []
labels_es = []
documents_es = []
punc_es = ['?', '!', ',', '.']

for intent in intents_es['intent']: 
    for sample in intent['samples']:
        word_list_es = nltk.word_tokenize(sample)
        words_es.extend(word_list_es)
        documents_es.append((word_list_es, intent['name']))
        if intent['name'] not in labels_es:
            labels_es.append(intent['name'])


In [7]:
# import NLP library
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

In [8]:
# turn words to lower case and to base form
words_en = [lem.lemmatize(word.lower()) for word in words_en if word not in punc_en]
words_es = [lem.lemmatize(word.lower()) for word in words_es if word not in punc_es]


# sort words and remove duplicates in English
words_en = sorted(set(words_en))
labels_en = sorted(set(labels_en))

# sort words and remove duplicates in Spanish
words_es = sorted(set(words_es))
labels_es = sorted(set(labels_es))


## Preprocessing input and output data  (Bag of Words)

Since machine learning models can not input strings, the text data needs to be converted into matrices prior to being used to train the models. 

In [9]:
# create training and output arrays in English
training_en = []
output_empty_en = [0] * len(labels_en)
for document in documents_en:
    bag_en = []
    word_patterns_en = document[0]
    word_patterns_en = [lem.lemmatize(word.lower()) for word in word_patterns_en]
    for word in words_en:
        if word in word_patterns_en:
            bag_en.append(1)
        else:
            bag_en.append(0)
            
    # copy output list
    output_en = list(output_empty_en)
    output_en[labels_en.index(document[1])] = 1
    training_en.append([bag_en, output_en])


In [10]:
# create training and output arrays in Spanish
training_es = []
output_empty_es = [0] * len(labels_es)
for document in documents_es:
    bag_es = []
    word_patterns_es = document[0]
    word_patterns_es = [lem.lemmatize(word.lower()) for word in word_patterns_es]
    for word in words_es:
        if word in word_patterns_es:
            bag_es.append(1)
        else:
            bag_es.append(0)
            
    # copy output list
    output_es = list(output_empty_es)
    output_es[labels_es.index(document[1])] = 1
    training_es.append([bag_es, output_es])


In [11]:
# randomizing the order of the training data in English
random.shuffle(training_en)
training_en = np.array(training_en)

train_x_en = list(training_en[:,0])
train_y_en = list(training_en[:,1])

# randomizing the order of the training data in Spanish
random.shuffle(training_es)
training_es = np.array(training_es)

train_x_es = list(training_es[:,0])
train_y_es = list(training_es[:,1])


## Build Model

The next step is to establish the architecture of the model that will predict user intent. There are approaches towards creating such a model. As long as the model registers a 100% accuracy, it will suffice. After experimenting with various architectures and a range of different parameters, the final predictive model is a 3-layer multilayer perceptron, which can be seen in the kernel below.

In [12]:
def model_en():
    model = Sequential()
    model.add(Dense(128, input_shape=(len(train_x_en[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(train_y_en[0]), activation='softmax'))
    return model


def model_es():
    model = Sequential()
    model.add(Dense(128, input_shape=(len(train_x_es[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(train_y_es[0]), activation='softmax'))
    return model


### Train English Model 

The English and Spanish model are trained with the English and Spanish data respectively. These models are trained for 500 Epochs, with the best models being saved in .h5 files. 

In [13]:
model_en = model_en()

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_en.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint_en = ModelCheckpoint(filepath='model_en.h5', monitor='acc')
model_en.fit(np.array(train_x_en), np.array(train_y_en), epochs=500, batch_size=5, verbose=1, callbacks=[checkpoint_en])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
93/93 [==============================] - 0s 3ms/sample - loss: 3.2134 - acc: 0.0215
Epoch 2/500
93/93 [==============================] - 0s 236us/sample - loss: 3.1260 - acc: 0.0753
Epoch 3/500
93/93 [==============================] - 0s 236us/sample - loss: 3.0857 - acc: 0.1398
Epoch 4/500
93/93 [==============================] - 0s 268us/sample - loss: 3.0212 - acc: 0.1183
Epoch 5/500
93/93 [==============================] - 0s 279us/sample - loss: 2.9375 - acc: 0.1720
Epoch 6/500
93/93 [==============================] - 0s 322us/sample - loss: 2.8459 - acc: 0.1720
Epoch 7/500
93/93 [==============================] - 0s 268us/sample - loss: 2.7432 - acc: 0.2043
Epoch 8/500
93/93 [==============================] - 0s 268us/sample - loss: 2.61

Epoch 74/500
93/93 [==============================] - 0s 247us/sample - loss: 0.1751 - acc: 0.9785
Epoch 75/500
93/93 [==============================] - 0s 236us/sample - loss: 0.1589 - acc: 0.9570
Epoch 76/500
93/93 [==============================] - 0s 236us/sample - loss: 0.3238 - acc: 0.8925
Epoch 77/500
93/93 [==============================] - 0s 236us/sample - loss: 0.1678 - acc: 0.9570
Epoch 78/500
93/93 [==============================] - 0s 247us/sample - loss: 0.1823 - acc: 0.9570
Epoch 79/500
93/93 [==============================] - 0s 247us/sample - loss: 0.2349 - acc: 0.9355
Epoch 80/500
93/93 [==============================] - 0s 257us/sample - loss: 0.2962 - acc: 0.8925
Epoch 81/500
93/93 [==============================] - 0s 247us/sample - loss: 0.1594 - acc: 0.9355
Epoch 82/500
93/93 [==============================] - 0s 268us/sample - loss: 0.1278 - acc: 0.9785
Epoch 83/500
93/93 [==============================] - 0s 236us/sample - loss: 0.1796 - acc: 0.9355
Epoch 84/5

93/93 [==============================] - 0s 268us/sample - loss: 0.0929 - acc: 0.9785
Epoch 156/500
93/93 [==============================] - 0s 257us/sample - loss: 0.0976 - acc: 0.9785
Epoch 157/500
93/93 [==============================] - 0s 290us/sample - loss: 0.1206 - acc: 0.9570
Epoch 158/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0565 - acc: 0.9892
Epoch 159/500
93/93 [==============================] - 0s 257us/sample - loss: 0.0547 - acc: 0.9892
Epoch 160/500
93/93 [==============================] - 0s 268us/sample - loss: 0.0900 - acc: 0.9570
Epoch 161/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0925 - acc: 0.9785
Epoch 162/500
93/93 [==============================] - 0s 268us/sample - loss: 0.0337 - acc: 1.0000
Epoch 163/500
93/93 [==============================] - 0s 290us/sample - loss: 0.1230 - acc: 0.9570
Epoch 164/500
93/93 [==============================] - 0s 247us/sample - loss: 0.1267 - acc: 0.9785
Epoch 165/500


93/93 [==============================] - 0s 365us/sample - loss: 0.0279 - acc: 1.0000
Epoch 238/500
93/93 [==============================] - 0s 268us/sample - loss: 0.0480 - acc: 0.9892
Epoch 239/500
93/93 [==============================] - 0s 611us/sample - loss: 0.0297 - acc: 0.9892
Epoch 240/500
93/93 [==============================] - 0s 322us/sample - loss: 0.0537 - acc: 0.9892
Epoch 241/500
93/93 [==============================] - 0s 300us/sample - loss: 0.0691 - acc: 0.9677
Epoch 242/500
93/93 [==============================] - 0s 236us/sample - loss: 0.0355 - acc: 0.9892
Epoch 243/500
93/93 [==============================] - 0s 332us/sample - loss: 0.0422 - acc: 0.9892
Epoch 244/500
93/93 [==============================] - 0s 268us/sample - loss: 0.1193 - acc: 0.9462
Epoch 245/500
93/93 [==============================] - 0s 268us/sample - loss: 0.0790 - acc: 0.9785
Epoch 246/500
93/93 [==============================] - 0s 257us/sample - loss: 0.0219 - acc: 1.0000
Epoch 247/500


93/93 [==============================] - 0s 257us/sample - loss: 0.0713 - acc: 0.9785
Epoch 320/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0649 - acc: 0.9892
Epoch 321/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0649 - acc: 0.9892
Epoch 322/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0196 - acc: 1.0000
Epoch 323/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0360 - acc: 0.9892
Epoch 324/500
93/93 [==============================] - 0s 236us/sample - loss: 0.0703 - acc: 0.9677
Epoch 325/500
93/93 [==============================] - 0s 236us/sample - loss: 0.0807 - acc: 0.9677
Epoch 326/500
93/93 [==============================] - 0s 236us/sample - loss: 0.0418 - acc: 1.0000
Epoch 327/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0480 - acc: 0.9892
Epoch 328/500
93/93 [==============================] - 0s 236us/sample - loss: 0.0551 - acc: 0.9892
Epoch 329/500


93/93 [==============================] - 0s 236us/sample - loss: 0.0404 - acc: 0.9785
Epoch 402/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0173 - acc: 0.9892
Epoch 403/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0141 - acc: 1.0000
Epoch 404/500
93/93 [==============================] - 0s 236us/sample - loss: 0.0119 - acc: 1.0000
Epoch 405/500
93/93 [==============================] - 0s 257us/sample - loss: 0.0257 - acc: 0.9892
Epoch 406/500
93/93 [==============================] - 0s 236us/sample - loss: 0.0300 - acc: 0.9892
Epoch 407/500
93/93 [==============================] - 0s 236us/sample - loss: 0.0173 - acc: 1.0000
Epoch 408/500
93/93 [==============================] - 0s 236us/sample - loss: 0.0664 - acc: 0.9785
Epoch 409/500
93/93 [==============================] - 0s 236us/sample - loss: 0.1015 - acc: 0.9785
Epoch 410/500
93/93 [==============================] - 0s 247us/sample - loss: 0.0277 - acc: 1.0000
Epoch 411/500


93/93 [==============================] - 0s 279us/sample - loss: 0.0187 - acc: 1.0000
Epoch 484/500
93/93 [==============================] - 0s 268us/sample - loss: 0.0176 - acc: 1.0000
Epoch 485/500
93/93 [==============================] - 0s 268us/sample - loss: 0.0200 - acc: 0.9892
Epoch 486/500
93/93 [==============================] - 0s 257us/sample - loss: 0.0180 - acc: 1.0000
Epoch 487/500
93/93 [==============================] - 0s 268us/sample - loss: 0.0208 - acc: 1.0000
Epoch 488/500
93/93 [==============================] - 0s 279us/sample - loss: 0.0082 - acc: 1.0000
Epoch 489/500
93/93 [==============================] - 0s 268us/sample - loss: 0.0277 - acc: 0.9892
Epoch 490/500
93/93 [==============================] - 0s 279us/sample - loss: 0.0223 - acc: 0.9892
Epoch 491/500
93/93 [==============================] - 0s 290us/sample - loss: 0.0378 - acc: 0.9892
Epoch 492/500
93/93 [==============================] - 0s 268us/sample - loss: 0.1072 - acc: 0.9677
Epoch 493/500


### Train Spanish Model 

In [14]:
model_es = model_es()

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_es.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint_es = ModelCheckpoint(filepath='model_es.h5', monitor='acc')
model_es.fit(np.array(train_x_es), np.array(train_y_es), epochs=500, batch_size=5, verbose=1, callbacks=[checkpoint_es])

Epoch 1/500
56/56 [==============================] - 0s 5ms/sample - loss: 3.1895 - acc: 0.0536
Epoch 2/500
56/56 [==============================] - 0s 481us/sample - loss: 3.1844 - acc: 0.1071
Epoch 3/500
56/56 [==============================] - 0s 410us/sample - loss: 3.1017 - acc: 0.0536
Epoch 4/500
56/56 [==============================] - 0s 445us/sample - loss: 3.0782 - acc: 0.0536
Epoch 5/500
56/56 [==============================] - 0s 427us/sample - loss: 2.9536 - acc: 0.1429
Epoch 6/500
56/56 [==============================] - 0s 481us/sample - loss: 2.9202 - acc: 0.1786
Epoch 7/500
56/56 [==============================] - 0s 499us/sample - loss: 2.8634 - acc: 0.1786
Epoch 8/500
56/56 [==============================] - 0s 445us/sample - loss: 2.7345 - acc: 0.2321
Epoch 9/500
56/56 [==============================] - 0s 392us/sample - loss: 2.6543 - acc: 0.2321
Epoch 10/500
56/56 [==============================] - 0s 481us/sample - loss: 2.5830 - acc: 0.3214
Epoch 11/500
56/56 [=

56/56 [==============================] - 0s 445us/sample - loss: 0.4353 - acc: 0.8393
Epoch 84/500
56/56 [==============================] - 0s 427us/sample - loss: 0.2907 - acc: 0.9286
Epoch 85/500
56/56 [==============================] - 0s 463us/sample - loss: 0.2570 - acc: 0.9286
Epoch 86/500
56/56 [==============================] - 0s 427us/sample - loss: 0.5701 - acc: 0.8393
Epoch 87/500
56/56 [==============================] - 0s 445us/sample - loss: 0.2289 - acc: 0.9643
Epoch 88/500
56/56 [==============================] - 0s 410us/sample - loss: 0.2434 - acc: 0.9464
Epoch 89/500
56/56 [==============================] - 0s 570us/sample - loss: 0.2506 - acc: 0.9464
Epoch 90/500
56/56 [==============================] - 0s 517us/sample - loss: 0.2743 - acc: 0.8929
Epoch 91/500
56/56 [==============================] - 0s 445us/sample - loss: 0.2730 - acc: 0.8929
Epoch 92/500
56/56 [==============================] - 0s 392us/sample - loss: 0.2362 - acc: 0.9643
Epoch 93/500
56/56 [===

56/56 [==============================] - 0s 321us/sample - loss: 0.2402 - acc: 0.9464
Epoch 165/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1640 - acc: 0.9464
Epoch 166/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1986 - acc: 0.9107
Epoch 167/500
56/56 [==============================] - 0s 321us/sample - loss: 0.2038 - acc: 0.9464
Epoch 168/500
56/56 [==============================] - 0s 338us/sample - loss: 0.2441 - acc: 0.9464
Epoch 169/500
56/56 [==============================] - 0s 321us/sample - loss: 0.2043 - acc: 0.9464
Epoch 170/500
56/56 [==============================] - 0s 338us/sample - loss: 0.2468 - acc: 0.9286
Epoch 171/500
56/56 [==============================] - 0s 338us/sample - loss: 0.2174 - acc: 0.9464
Epoch 172/500
56/56 [==============================] - 0s 321us/sample - loss: 0.2794 - acc: 0.8929
Epoch 173/500
56/56 [==============================] - 0s 320us/sample - loss: 0.1216 - acc: 0.9643
Epoch 174/500


56/56 [==============================] - 0s 338us/sample - loss: 0.1293 - acc: 0.9286
Epoch 247/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0637 - acc: 0.9821
Epoch 248/500
56/56 [==============================] - 0s 338us/sample - loss: 0.0704 - acc: 0.9821
Epoch 249/500
56/56 [==============================] - 0s 338us/sample - loss: 0.1514 - acc: 0.9643
Epoch 250/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0474 - acc: 0.9821
Epoch 251/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1040 - acc: 0.9643
Epoch 252/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1974 - acc: 0.9107
Epoch 253/500
56/56 [==============================] - 0s 303us/sample - loss: 0.1187 - acc: 0.9643
Epoch 254/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1808 - acc: 0.9643
Epoch 255/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0820 - acc: 0.9821
Epoch 256/500


Epoch 328/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1613 - acc: 0.9464
Epoch 329/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0347 - acc: 0.9821
Epoch 330/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1161 - acc: 0.9821
Epoch 331/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1069 - acc: 0.9464
Epoch 332/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1342 - acc: 0.9464
Epoch 333/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0563 - acc: 0.9821
Epoch 334/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0576 - acc: 1.0000
Epoch 335/500
56/56 [==============================] - 0s 588us/sample - loss: 0.0712 - acc: 0.9821
Epoch 336/500
56/56 [==============================] - 0s 338us/sample - loss: 0.0809 - acc: 0.9643
Epoch 337/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0441 - acc: 0.9821


Epoch 410/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1440 - acc: 0.9286
Epoch 411/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0984 - acc: 0.9464
Epoch 412/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1415 - acc: 0.9464
Epoch 413/500
56/56 [==============================] - 0s 321us/sample - loss: 0.1000 - acc: 0.9643
Epoch 414/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0251 - acc: 1.0000
Epoch 415/500
56/56 [==============================] - 0s 320us/sample - loss: 0.0944 - acc: 0.9286
Epoch 416/500
56/56 [==============================] - 0s 303us/sample - loss: 0.0511 - acc: 0.9643
Epoch 417/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0876 - acc: 0.9643
Epoch 418/500
56/56 [==============================] - 0s 303us/sample - loss: 0.1654 - acc: 0.9286
Epoch 419/500
56/56 [==============================] - 0s 320us/sample - loss: 0.1039 - acc: 0.9286


Epoch 492/500
56/56 [==============================] - 0s 410us/sample - loss: 0.0687 - acc: 0.9643
Epoch 493/500
56/56 [==============================] - 0s 392us/sample - loss: 0.1356 - acc: 0.9464
Epoch 494/500
56/56 [==============================] - 0s 410us/sample - loss: 0.0600 - acc: 0.9821
Epoch 495/500
56/56 [==============================] - 0s 374us/sample - loss: 0.0863 - acc: 0.9643
Epoch 496/500
56/56 [==============================] - 0s 445us/sample - loss: 0.0781 - acc: 0.9643
Epoch 497/500
56/56 [==============================] - 0s 428us/sample - loss: 0.1393 - acc: 0.9286
Epoch 498/500
56/56 [==============================] - 0s 392us/sample - loss: 0.0733 - acc: 0.9821
Epoch 499/500
56/56 [==============================] - 0s 338us/sample - loss: 0.0503 - acc: 0.9821
Epoch 500/500
56/56 [==============================] - 0s 321us/sample - loss: 0.0782 - acc: 0.9821


## Preprocess User Query

Before testing the models, it is necessary for the user queries to undergo the same preprocessing as the data used to train the model. Otherwise, the models will not be able to input the data. 

In [15]:
# performs preprocessing on inputted text (lower casing and lemmatization)
def preprocess_input(text):
    text_token = nltk.word_tokenize(text)
    text_token = [lem.lemmatize(word.lower()) for word in text_token]
    return text_token

# coverts preprocessed text to a matrix form 
def bow_en(text_token):
    bag = []
    for word in words_en:
        if word in text_token:
            bag.append(1)
        else:
            bag.append(0)
    return bag

def bow_es(text_token):
    bag = []
    for word in words_es:
        if word in text_token:
            bag.append(1)
        else:
            bag.append(0)
    return bag

# inputs user query in matrix form and returns a predicted label 
def predict_label_en(bow_en):
    prediction = model_en.predict(np.array(bow_en).reshape((1, len(words_en))))
    max_value = np.max(prediction)
    if max_value > 0.9:
        predict_label = labels_en[np.argmax(prediction)]
    else:
        predict_label = 'NA'
    return predict_label

def predict_label_es(bow_es):
    prediction = model_es.predict(np.array(bow_es).reshape((1, len(words_es))))
    max_value = np.max(prediction)
    if max_value > 0.9:
        predict_label = labels_es[np.argmax(prediction)]
    else:
        predict_label = 'NA'
    return predict_label


## Response Generation

Once the model identifies the intent of a user query, it has to return the appropriate response. The functions below perform that role for English and Spanish queries. 

### English Responses

In [16]:
# inputs the user intent and returns the appropriate response
def generate_response_en(query_intent, user):
    
    
    responses_en = { 
        'cancel' : "Ok, it's cancelled. Once you're ready, please type *`daily`* to fill out your follow-up reports.",

        'chicken' : "Here comes Arthur!\n\n*[Launch Rubber Chicken]({{{url}}})* 🐔\n\n_Pro tip: you can change the last part of Arthur's URL in your browser in order to create a unique room.",

        'daily' :  "No response",
            
        'dailystatus' : "No response",
            
        'dashboard' :"Ok, please *[follow this link]({{{url}}})* to access the web dashboard.",

        'datetime' :  "The time information for your user is:",
            
        'disable' : "Ok, it's done. You have disconnected and disabled all private messaging between you and me.\nYou can still use the [web application]({{{url}}}).\n\nIf you want to talk to me again, just type *`enable`*",


        'enable' : ["It seems that you had enabled me previously, I'm ready to help you.\nPlease type *`help`* to know more about how I can help you.",
                    
                    "Hey! Welcome back, I'm ready to help you.\nYou can *[login into the web application]({{{url}}})* to check your settings."],
            
        'extras' :"Hey, your wish is my command, and these are the extra commands",
            
        'feedback' : "Hey {}, it'd be great to get some feedback from you. If it's something important, I recommend you to visit our support center.".format(user.first_name),

        'hello' : "Hey {}! Have a great day and remember I'm so happy about being your friend.\nType *`help`* to learn more about me.".format(user.first_name),

        'help' : "Hey, your wish is my command, and these are my available commands:",

        'kudos' :  "kudos response: ",
        
        'news' :  "Take a look at *[my blog](https://www.dailybot.co/blog/)* to read about tips, key learnings and other content that our team is sharing.",

        'report' : 'No response',
        
        'resetdaily' :"This command has been changed, please type *`help`* to see all current commands.",

        'settings' : "This is a quick overview of your settings. You can manage these settings on my *[Web Application]({{{url}}})*:",

        'skipdaily' :"no response",
        
        'snooze' : "Ok, I’ll remind you in *{{time}} minutes*",

        'textformat' : "Hi there! The following is a list of the different formats I support.\nYou can also see this *[support article](https://www.dailybot.com)* to know more about how to build responses with these formats.",

        'thanks' : ["Oh, don't you even mention it. I'm always happy to help.",
                                "It's always my pleasure.",
                                "You are so very welcome.",
                                "It's no trouble at all. 😊",
                                "Anytime! it's always my pleasure. 😊"],
       
        'timezone' : "The time information for the typed timezone is:",

        'NA':  "Sorry for not understanding. Please type *`help`* for me to help you.",
        
        'meeting': "Set an appointment with whom?"
        
        
    }

    return responses_en[query_intent]


### Spanish Responses

In [17]:
    
# inputs the user intent and returns the appropriate response
def generate_response_es(query_intent, user):
    
    responses_es = { 
        'cancel' : "Ok, has cancelado tu seguimiento diario. Por favor escribe *`daily`* para empezar cuando desees.",
    
        'chicken' : "¡Aquí viene Arthur!\n\n[Abrir Rubber Chicken]({{{url}}}) 🐔\n\n_Pro tip: puedes cambiar la última parte de la URL de Arthur en tu navegador y así tendrás una sala única._",

        'daily' :  "No hay respuesta",
        
        'dailystatus' :  "No hay respuesta",
        
        'dashboard' : "Ok, *[abre este enlace]({{{url}}})* para abrir la web y dashboard.",

        'datetime' : "La información horaria de tu usuario es:",

        'disable' : "Listo, está hecho. Has desconectado y deshabilitado todos los mensajes privados entre tu y yo.\nPor cierto, puedes seguir utilizando mi [aplicación web]({{{url}}}).\n\nSi deseas volver a hablar conmigo simplemente escribe *`enable`*.",

        'enable' : "Parece que ya me habías habilitado previamente, estoy listo para ayudarte.\nPor favor, escribe *`help`* para saber más sobre cómo puedo ayudarte.",
                    # 8. "¡Hola! Bienvenido de nuevo, estoy listo para ayudarte.\nPor favor inicia sesión en mi *[aplicación web]({{{url}}})* para confirmar su configuración."
        
        'extras' : "Hola, puedes usar estos comandos extras para interactuar conmigo:",

        
        'feedback' : "Hola {}!, Sería grandióso recibir tu feedback. Si es algo importante, Te recomiendo visitar nuestro centro de soporte.".format(user.first_name),

        
        'hello' :  "¡Hola {}! Ten un excelente día y recuerda que estoy muy feliz de ser tu amigo.\nEscribe *`help`* para aprender más sobre mí.".format(user.first_name),

        
        'help' : "Hola, puedes usar estos comandos para interactuar conmigo:",
        
        'kudos' :"kudos response: ",
        
        'news' : "Echa un vistazo a *[mi blog](https://www.dailybot.com/blog/)* (en inglés) para leer sobre trucos, aprendizajes y otro contenido que comparte nuestro equipo.",

        
        'report' : "No hay respuesta",
        
        'resetdaily' : "Este comando ha cambiado, por favor escribe *`help`* para ver todos los comandos actuales.",

        'settings' : "Este es un resúmen de tu configuración. Puedes gestionar la configuración en mi *[aplicación web]({{{url}}})*:",

        'skipdaily' : "No hay respuesta",
        
        'snooze' : "Ok, te recordaré en *{{time}} minutos*.",

        'textformat' : "Hola! La siguiente es una lista de los diferentes formatos que soporto.\nPuedes verlo aquí *[artículo de apoyo](https://www.dailybot.com)* para conocer más acerca de como construir respuestas con estos formatos.",

        'thanks' : ["¡Con muuucho gusto!",
                                 "Siempre es un placer.",
                                 "Estoy para ayudarte.",
                                 "A ti por tu tiempo. 😊",
                                 "¡Cuando me necesites! siempre es un placer. 😊"],
    
        'timezone' :  "La información de la zona horaria introducida es:",

        'NA': "Lo siento por no entender. Por favor escribe *`help`* para ayudarte.",
        
        'meeting': "Set an appointment with whom?"
    }

    return responses_es[query_intent]

## Saving unidentified Queries

A feature included in the bot is to save queries that are unidentified by the chabot. The bot will mainly fail to identify an intent of a user query if the query is poorly written or if it contains data that was not used when training the model. Storing unidentified queries will enable Dailybot to see what text the chatbot is struggling to deal with and what modifications the model may need. 

In [18]:
# Create list of queries with no response
try:
    no_intent_queries = open("no_intent_queries.txt", "x")
    queries_list = ""

except:
    file = open("no_intent_queries.txt","r+")
    file.truncate(0)
    file.close()
    queries_list = ""




## Load User Object

For the sake of evaluating the chatbots ability to incorporate user data into its responses, fake data of users is created with classes in a separate .py file. In testing the chatbot, two user objects will be created, one representing Aashish and the other representing Sergio. The "aashish_user" object will test the English model, whereas the "sergio_user" object will test the Spanish model. 

In [19]:
from imp import reload
import UserObjects

reload(UserObjects)

User = UserObjects.User
Settings = UserObjects.Settings
DailyReport = UserObjects.DailyReport
Appointment = UserObjects.Appointment
vars(Appointment)


mappingproxy({'__module__': 'UserObjects',
              '__init__': <function UserObjects.Appointment.__init__(self, member, time)>,
              'show_appointments': <function UserObjects.Appointment.show_appointments(self)>,
              '__dict__': <attribute '__dict__' of 'Appointment' objects>,
              '__weakref__': <attribute '__weakref__' of 'Appointment' objects>,
              '__doc__': None})

In [20]:
aashish_user = User(first_name="Aashish", last_name="Nair", language="English")
aashish_settings = Settings(is_enabled=True, given_feedback=False)
aashish_daily_report = DailyReport(is_completed=True)
aashish_meeting = Appointment(member=None, time=None)

aashish_user.settings = aashish_settings
aashish_user.daily_report = aashish_daily_report
aashish_user.appointment = aashish_meeting


In [21]:
sergio_user = User(first_name="Sergio", last_name="Alex", language="Spanish")
sergio_settings = Settings(is_enabled=True, given_feedback=False)
sergio_daily_report = DailyReport(is_completed=True)
sergio_meeting = Appointment(member=None, time=None)

sergio_user.settings = sergio_settings
sergio_user.daily_report = sergio_daily_report
sergio_user.appointment = sergio_meeting



# Chatbot

All the steps in creating the chatbot, from preprocessing to data modeling, culminates in the following function. This function will ask for user input and return the suitable response. The chatbot will operate continuously until the user enters "quit" as the input.

In [22]:
def chatbot(user):
    
    
    queries_list = ""
    
    print('Insert command here (type "quit" to stop chatbot)\n')

    while 1 >0:
        
        input_text = input()
        
        # chatbot continues to accept queries until user quits
        if input_text=='quit':
            return queries_list
        
        # only inputs of 3 characters or more are accepted
        if len(input_text) < 3:
            while len(input_text) <3:
                input_text = input_text + " "
                
                
#----------------------------------------------------- English Bot -----------------------------------------------------                


        if user.language =='English':

            spell = Speller(lang='en')
            if input_text not in intents_en:
                input_text = spell(input_text)
                
            prediction_en = predict_label_en(np.array(bow_en(preprocess_input(input_text))))

            # thanks intent
            if prediction_en == 'thanks':
                response_en = random.choice(generate_response_en(prediction_en, user))
                print(response_en)
                print(' ')

            # enable intent
            elif prediction_en == 'enable':
                response_en = generate_response_en(prediction_en, user)
                if user.settings.is_enabled: 
                    response_en = response_en[0]
                else:
                    user.settings.enable()
                    response_en = response_en[1]
                print(response_en)
                print(' ')

            # disable intent                   
            elif prediction_en == 'disable':
                user.settings.disable()
                response_en = generate_response_en(prediction_en, user)
                print(response_en)
                print(' ')

            elif prediction_en == 'skipdaily':
                response_en = generate_response_en(prediction_en, user)
                user.daily_report.skip_daily()
                print(response_en)
                print(' ')

            elif prediction_en == 'cancel':
                response_en = generate_response_en(prediction_en, user)
                user.daily_report.cancel()
                print(response_en)
                print(' ')

            elif prediction_en == 'meeting':
                response_en = generate_response_en(prediction_en, user)
                print("Bot: ",response_en)
                print(' ')
                appointment_with = input()
                print('Which day?')
                time = ""
                appointment_day = input()
                time = time + appointment_day + " "
                print('What time?')
                appointment_time = input()
                time = time + appointment_time
                
                date = dateparser.parse(time)
                user.appointment = Appointment(member=appointment_with, time=date)
                
                print('Appointment scheduled at {} on {} with the following people: {}'.format(appointment_day, appointment_time, appointment_with))
                print(' ')
                
            elif prediction_en == 'meeting status':
                if vars(user.appointment)['time'] is None:
                    print('You have no meetings')
                else:
                    user.appointment.show_appointments()
                
            else:
                response_en = generate_response_en(prediction_en, user)
                print(response_en)
                print(' ')
            if prediction_en == 'NA':
                queries_list = queries_list + input_text+ '\n'
                
                
                
#----------------------------------------------------- Spanish Bot -----------------------------------------------------                

        elif user.language == 'Spanish':
        
            spell = Speller(lang='es')
            if input_text not in intents_en:
                input_text = spell(input_text)
           
            prediction_es = predict_label_es(np.array(bow_es(preprocess_input(input_text))))

            # thanks intent
            if prediction_es == 'thanks':
                response_es = random.choice(generate_response_es(prediction_es, user))
                print(response_es)
                print(' ')

            # enable intent
            elif prediction_es == 'enable':
                response_es = generate_response_es(prediction_es, user)
                if user.settings.is_enabled: 
                    response_es = response_es[0]
                else:
                    user.settings.enable()
                    response_es = response_es[1]
                print(response_es)
                print(' ')

            # disable intent                   
            elif prediction_es == 'disable':
                user.settings.disable()
                response_es = generate_response_es(prediction_es, user)
                print(response_es)
                print(' ')

            elif prediction_es == 'skipdaily':
                response_es = generate_response_es(prediction_es, user)
                user.daily_report.skip_daily()
                print(response_es)
                print(' ')

            elif prediction_es == 'cancel':
                response_es = generate_response_es(prediction_es, user)
                user.daily_report.cancel()
                print(response_es)
                print(' ')

            elif prediction_es == 'meeting':
                response_es = generate_response_es(prediction_es, user)
                print("Bot: ",response_es)
                print(' ')
                appointment_with = input()
                print('Which day?')
                time = ""
                appointment_day = input()
                time = time + appointment_day + " "
                print('What time?')
                appointment_time = input()
                time = time + appointment_time
                
                date = dateparser.parse(time)
                user.appointment = Appointment(member=appointment_with, time=date)
                
                print('Appointment scheduled at {} on {} with the following people: {}'.format(appointment_day, appointment_time, appointment_with))
                print(' ')
                
            elif prediction_es == 'meeting status':
                if vars(user.appointment)['time'] is None:
                    print('You have no meetings')
                else:
                    user.appointment.show_appointments()
                
            else:
                response_es = generate_response_es(prediction_es, user)
                print(response_es)
                print(' ')
            if prediction_es == 'NA':
                queries_list = queries_list + " " + input_text
                


In [23]:
queries_list = chatbot(aashish_user)

Insert command here (type "quit" to stop chatbot)

hi
Hey Aashish! Have a great day and remember I'm so happy about being your friend.
Type *`help`* to learn more about me.
 
hello
Hey Aashish! Have a great day and remember I'm so happy about being your friend.
Type *`help`* to learn more about me.
 
thanks
It's always my pleasure.
 
please cancel
Ok, it's cancelled. Once you're ready, please type *`daily`* to fill out your follow-up reports.
 
feedback, ok?
Hey Aashish, it'd be great to get some feedback from you. If it's something important, I recommend you to visit our support center.
 
chicken
Here comes Arthur!

*[Launch Rubber Chicken]({{{url}}})* 🐔

_Pro tip: you can change the last part of Arthur's URL in your browser in order to create a unique room.
 
disable
Ok, it's done. You have disconnected and disabled all private messaging between you and me.
You can still use the [web application]({{{url}}}).

If you want to talk to me again, just type *`enable`*
 
hola
Sorry for not 

## Add "NA" Queries to .txt File

In [24]:
file = open('no_intent_queries.txt', 'a')
# Append 'hello' at the end of file
file.write(queries_list)
# Close the file
file.close()

# Conclusion

In summation, the currently built microservice provides a following services:

    - The capability to identify intent based on user query
    - A polished response generator that takes into account user information
    - the ability to process English and Spanish queries
    - A .txt file that stores unidentified user queries for future inspection
    - A hypothetical "appointment" intent that can arrange meetings between users
    
It is, however, lacking in these aspects:

    - The Spanish data isn't as comprehensive as the english data (compare json files for an understanding of this)
    - The user objects are only an imitation and do not meet DailyBot's standards
    - The appointment intent can not be modified until its scope will clarified
    